In [1]:
!pip install boto3

In [2]:
!pip install langchain

  Using cached langchain-0.0.348-py3-none-any.whl (2.0 MB)
  Using cached langsmith-0.0.69-py3-none-any.whl (48 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached dataclasses_json-0.6.3-py3-none-any.whl (28 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached pydantic-2.5.2-py3-none-any.whl (381 kB)
  Using cached langchain_core-0.0.12-py3-none-any.whl (181 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached marshmallow-3.20.1-py3-none-any.whl (49 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Unin

In [3]:
!pip install llama-index

  Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached aiostream-0.5.2-py3-none-any.whl (39 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached nest_asyncio-1.5.8-py3-none-any.whl (5.3 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached fsspec-2023.12.1-py3-none-any.whl (168 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.4.22
    Uninstalling SQLAlchemy-1.4.22:
      Successfully uninstalled SQLAlchemy-1.4.22
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
  Attempting uninstall: regex
    Found existi

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
